<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/Integrador_M2_TATETI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El siguiente programa consiste en un juego de **TaTeTi** en el que el jugador se enfrentará a la máquina.

# Funcionamiento del programa:
El jugador seleccionará el número de partidas que desea jugar contra la máquina. El mínimo de partidas será de 1 y el máximo será de 5.

Luego, el jugador elige la dificultad.
*   F - Dificultad fácil. La máquina simplemente irá llenando espacios en el tablero de forma aleatoria.
*   N - Dificultad normal. La máquina intentará buscar una forma de ganar, y en caso de no poder hacerlo, intentará bloquear al jugador para que no pueda ganar.

Una vez seleccionadas las partidas y la dificultad, empiezan las partidas. El jugador siempre será el primero en iniciar. Se le presentará el tablero y tendrá que elegir una de las posiciones disponibles para poder insertar una 'X' o una 'O' (irán alternando al cambiar de partida, pero el jugador siempre será el primero en iniciar).

Cuando termina una partida, el tablero se reinicia y comienza una nueva hasta que se alcanza el número de partidas máximo. Una vez llegado este punto, el juego termina. Los resultados de cada juego se guardarán en un archivo diferente. El jugador tiene la opción de ingresar una 'S' para ver el registro histório de sus partidas o bien ingresar cualquier otro caracter para salir. En caso de presionar una 'S', se mostrarán todos los archivos generados con el resumen. Indicando cuantas partidas ganó el jugador, cuantas ganó la máquina, cuantas terminaron en empate y el porcentaje de cada una en cada juego.

# Contenidos de la materia:

*   Procesos pesados - El programa crea dos procesos emparentados. El padre controlará la lógica del juego y el hijo irá generando los archivos con las estadísticas de cada juego.
*   Threads - El programa creará un thread para visualizar el contenido de cada archivo de historial.
*   Comunicación - El proceso padre y el hijo se comunicarán por medio de Pipes. El padre le informará al hijo el ganador de una partida, o si las partidas han finalizado.
*   Sincronización - Los hilos están sincronizados por medio de Lock, para evitar que escriban en la consola al mismo tiempo. A medida que van llegando, irán mostrando sus historiales.

Generamos el archivo de python.

In [ ]:
%%writefile integrador.py
import os
import sys
import random
import threading
import glob
from multiprocessing import Pipe, Process

CONTADOR = 0
POSICION_0_TABLERO = 0
POSICION_1_TABLERO = 1
POSICION_2_TABLERO = 2
POSICION_3_TABLERO = 3
POSICION_4_TABLERO = 4
POSICION_5_TABLERO = 5
POSICION_6_TABLERO = 6
POSICION_7_TABLERO = 7
POSICION_8_TABLERO = 8
DIMENSION_TABLERO = 9
TAMANIO_HILERA = 3
PASAR_A_PORCENTAJE = 100
POSICION_CONTIGUA = 1
DOS_POSICIONES_CONTIGUAS = 2
POSICION_INFERIOR = 3
DOS_POSICIONES_INFERIORES = 6
GANO_JUGADOR = 1
GANO_MAQUINA = 2
EMPATE = 3
FIN_PARTIDA = 0
PARTIDAS_MINIMAS = 1
PARTIDAS_MAXIMAS = 5
DIFICULTAD_FACIL = 'f'
DIFICULTAD_NORMAL = 'n'
VER_HISTORIAL = 's'
NOMBRE_PARA_HISTORIALES = 'historial_partida.log'
ARCHIVOS_LOG = '*.log'
MODO_FACIL_ACTIVO = 4
MODO_NORMAL_ACTIVO = 5
INICIALIZADOR = 0
PROCESO_HIJO = 0
INCREMENTADOR = 1
IDENTIFICADOR_INICIAL_JUGADOR = 'X'
IDENTIFICADOR_INICIAL_MAQUINA = 'O'
AJUSTE_POSICION = 1

#Con esta función creo el nombre para los archivos de log.
def verificar_archivo(nombre_archivo):
    contador = CONTADOR
    nuevo_nombre = nombre_archivo
    while os.path.exists(nuevo_nombre):
        contador += INCREMENTADOR
        base_nombre, extension = os.path.splitext(nombre_archivo)
        nuevo_nombre = f"{base_nombre}_{contador}{extension}"
    return nuevo_nombre

#Funcion para crear el tablero. La usaré para reiniciarlo al inicio de cada partida
def crear_tablero():
    tablero = [' '] * DIMENSION_TABLERO
    return tablero

#Crear un objeto Lock
lock = threading.Lock()

#Con esta función leo el contenido de un archivo
def leer_archivo(archivo):
    # Adquirir el bloqueo
    lock.acquire()
    try:
        with open(archivo, 'r') as f:
            contenido = f.read()
        print(f"Contenido del archivo {archivo}:")
        print(contenido)
    finally:
        print("")
        # Cerrar el archivo
        f.close()
        # Liberar el bloqueo
        lock.release()

#Con esta función muestro el tablero
def imprimir_tablero(tablero):
    print('                                        Referencia de posiciones a seleccionar ')
    print(' ' + tablero[POSICION_0_TABLERO] + ' | ' + tablero[POSICION_1_TABLERO] + ' | ' + tablero[POSICION_2_TABLERO] + '                                           1 | 2 | 3 ')
    print('---+---+---                                         ---+---+--- ')
    print(' ' + tablero[POSICION_3_TABLERO] + ' | ' + tablero[POSICION_4_TABLERO] + ' | ' + tablero[POSICION_5_TABLERO] + '                                           4 | 5 | 6 ')
    print('---+---+---                                         ---+---+--- ')
    print(' ' + tablero[POSICION_6_TABLERO] + ' | ' + tablero[POSICION_7_TABLERO] + ' | ' + tablero[POSICION_8_TABLERO] + '                                           7 | 8 | 9 ')

#Con esta función me fijo si el jugador o la maquina ganaron
def hay_ganador(tablero):
    # Verificar filas
    for i in range(POSICION_0_TABLERO, DIMENSION_TABLERO, TAMANIO_HILERA):
        if tablero[i] == tablero[i + POSICION_CONTIGUA] == tablero[i + DOS_POSICIONES_CONTIGUAS] != ' ':
            return True

    # Verificar columnas
    for i in range(TAMANIO_HILERA):
        if tablero[i] == tablero[i + POSICION_INFERIOR] == tablero[i + DOS_POSICIONES_INFERIORES] != ' ':
            return True

    # Verificar diagonales
    if tablero[POSICION_0_TABLERO] == tablero[POSICION_4_TABLERO] == tablero[POSICION_8_TABLERO] != ' ' or tablero[POSICION_2_TABLERO] == tablero[POSICION_4_TABLERO] == tablero[POSICION_6_TABLERO] != ' ':
        return True

    return False

#Esta función controla la lógica de la máquina
def jugada_maquina(tablero, modo_facil, maquina, jugador):
    posiciones_disponibles = [i for i, c in enumerate(tablero) if c == ' ']

    if not modo_facil:
        #Si puse modo normal, la máquina intentará ganar
        for posicion in posiciones_disponibles:
            tablero[posicion] = maquina
            if hay_ganador(tablero):
                return

            tablero[posicion] = ' '

        #Si puse modo normal, la máquina verifica si el jugador puede ganar en el siguiente movimiento y le bloqueará la jugada
        for posicion in posiciones_disponibles:
            tablero[posicion] = jugador
            if hay_ganador(tablero):
                tablero[posicion] = maquina
                return

            tablero[posicion] = ' '

    #La máquina elige una posicion aleatoria en caso de no poder ganar ni bloquear al jugador. O si está en modo fácil.
    posicion = random.choice(posiciones_disponibles)
    tablero[posicion] = maquina

#Proceso hijo.
def proceso_hijo(pipe):
    partidas_jugadas = partidas_ganadas_jugador = partidas_ganadas_maquina = partidas_empate = INICIALIZADOR

    while True:
        resultado = pipe.recv()
        if resultado == FIN_PARTIDA:
            total_partidas = partidas_jugadas
            porcentaje_victorias_jugador = (partidas_ganadas_jugador / total_partidas) * PASAR_A_PORCENTAJE
            porcentaje_victorias_maquina = (partidas_ganadas_maquina / total_partidas) * PASAR_A_PORCENTAJE
            porcentaje_empates = (partidas_empate / total_partidas) * PASAR_A_PORCENTAJE
            print('Porcentaje de victorias del jugador:', porcentaje_victorias_jugador)
            print('Porcentaje de victorias de la máquina:', porcentaje_victorias_maquina)
            print('Porcentaje de partidas empatadas:', porcentaje_empates)
            break
        elif resultado == GANO_JUGADOR:
            partidas_ganadas_jugador += INCREMENTADOR
            partidas_jugadas += INCREMENTADOR
            print('Partida', partidas_jugadas,'Ganada por jugador')
        elif resultado == GANO_MAQUINA:
            partidas_ganadas_maquina += INCREMENTADOR
            partidas_jugadas += INCREMENTADOR
            print('Partida', partidas_jugadas,'Ganada por maquina')
        elif resultado == EMPATE:
            partidas_empate += INCREMENTADOR
            partidas_jugadas += INCREMENTADOR
            print('Partida', partidas_jugadas,'Termino en empate')
        elif resultado == MODO_FACIL_ACTIVO:
            print('DIFICULTAD: FACIL')
        elif resultado == MODO_NORMAL_ACTIVO:
            print('DIFICULTAD: NORMAL')


#Proceso padre
def proceso_padre(pipe):
    #Declara variables para el juego
    jugador = IDENTIFICADOR_INICIAL_JUGADOR
    maquina = IDENTIFICADOR_INICIAL_MAQUINA

    #Pide las partidas maximas
    while True:
        try:
          max_partidas = int(input('Ingrese la cantidad de partidas a jugar. Debe estar entre {} y {}: '.format(PARTIDAS_MINIMAS, PARTIDAS_MAXIMAS)))
          if max_partidas >= PARTIDAS_MINIMAS and max_partidas <= PARTIDAS_MAXIMAS:
              break
          else:
              print("El número de partidas ingresado no es correcto. Intente nuevamente.")
        except ValueError:
          print(f"El caracter ingresado no es correcto. Recuerde que debe ser un número del {PARTIDAS_MINIMAS} al {PARTIDAS_MAXIMAS}. Intente nuevamente.")

    #Pide el nivel de dificultad
    modo_facil = False
    while True:
        dificultad = input("Ingrese la dificultad ('F' para fácil, 'N' para normal): ")
        if dificultad.casefold() == DIFICULTAD_FACIL:
            modo_facil = True
            break
        elif dificultad.casefold() == DIFICULTAD_NORMAL:
            modo_facil = False
            break
        else:
            print("Dificultad inválida. Intente nuevamente.")

    if modo_facil:
        pipe.send(MODO_FACIL_ACTIVO)
    else:
        pipe.send(MODO_NORMAL_ACTIVO)

    for partida in range(max_partidas):
        print('Usted juega con:', jugador)
        #Se reinicia el tablero
        tablero = crear_tablero()

        while True:
            imprimir_tablero(tablero)

            #Es el turno del jugador
            try:
              posicion = int(input('Ingrese la posición deseada (1 a 9): '))
              if posicion > POSICION_0_TABLERO and posicion <= DIMENSION_TABLERO and tablero[posicion-AJUSTE_POSICION] == ' ':
                  tablero[posicion-AJUSTE_POSICION] = jugador
              else:
                  print('Posición inválida. Intente nuevamente.')
                  continue
            except ValueError:
                  print(f"Posición inválida. Recuerde que debe ser un numero del 1 al {DIMENSION_TABLERO}. Intente nuevamente.")
                  continue

            #Comprueba si el jugador ganó
            if hay_ganador(tablero):
                imprimir_tablero(tablero)
                print('¡Has ganado!')
                pipe.send(GANO_JUGADOR)
                break

            #Comprueba si hubo empate
            if ' ' not in tablero:
                imprimir_tablero(tablero)
                print('¡Empate!')
                pipe.send(EMPATE)
                break

            #Es el turno de la máquina
            jugada_maquina(tablero, modo_facil, maquina, jugador)

            #Comprueba si la máquina ganó
            if hay_ganador(tablero):
                imprimir_tablero(tablero)
                print('La máquina ha ganado.')
                pipe.send(GANO_MAQUINA)
                break
        #Intercambio los caracteres del jugador y la maquina
        jugador, maquina = maquina, jugador

    #Le avisa al hijo que acabaron las partidas.
    pipe.send(FIN_PARTIDA)

if __name__ == '__main__':
    # Crear los pipes para la comunicación entre procesos
    padre_pipe, hijo_pipe = Pipe()

    pid = os.fork()

    if pid > PROCESO_HIJO:
        #Proceso padre
        hijo_pipe.close()
        proceso_padre(padre_pipe)
        padre_pipe.close()
        os.wait()

        ver_historial = input('¿Desea ver el historial de partidas? (\'S\' para aceptar, cualquier otro caracter lo hara salir): ')
        if ver_historial.casefold() == VER_HISTORIAL:
            print("")
            print("Historial de partidas")
            print("")
            archivos = glob.glob(ARCHIVOS_LOG)

            hilos = []
            for archivo in archivos:
              hilo = threading.Thread(target=leer_archivo, args=(archivo,))
              hilo.start()
              hilos.append(hilo)

            #Espera a que todos los hilos terminen su ejecucion
            for hilo in hilos:
              hilo.join()

    elif pid == PROCESO_HIJO:
        #Proceso hijo
        sys.stdout = open(verificar_archivo(NOMBRE_PARA_HISTORIALES), 'w')
        padre_pipe.close()
        proceso_hijo(hijo_pipe)
        hijo_pipe.close()
        sys.stdout.close()
        sys.exit()

    else:
        print('Ha ocurrido un error al crear el proceso de generacion de historiales.')

Writing integrador.py


Listamos los archivos para ver que efectivamente se haya creado.

In [ ]:
!ls -la

total 28
drwxr-xr-x 1 root root  4096 Jul  5 19:47 .
drwxr-xr-x 1 root root  4096 Jul  5 19:45 ..
drwxr-xr-x 4 root root  4096 Jun 30 13:33 .config
-rw-r--r-- 1 root root 10640 Jul  5 19:46 integrador.py
drwxr-xr-x 1 root root  4096 Jun 30 13:34 sample_data


Ejecutamos el programa. Podemos elegir si deseamos o no ver el historial.

In [ ]:
!python3 integrador.py

Ingrese la cantidad de partidas a jugar. Debe estar entre 1 y 5: 2
Ingrese la dificultad ('F' para fácil, 'N' para normal): N
Usted juega con: X
                                        Referencia de posiciones a seleccionar 
   |   |                                             1 | 2 | 3 
---+---+---                                         ---+---+--- 
   |   |                                             4 | 5 | 6 
---+---+---                                         ---+---+--- 
   |   |                                             7 | 8 | 9 
Ingrese la posición deseada (1 a 9): 2
                                        Referencia de posiciones a seleccionar 
   | X | O                                           1 | 2 | 3 
---+---+---                                         ---+---+--- 
   |   |                                             4 | 5 | 6 
---+---+---                                         ---+---+--- 
   |   |                                             7 | 8 | 9 
Ingrese la p